In [ ]:
import cv2
import matplotlib
import numpy as np
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

img_color = cv2.imread("img.jpg")
img_gray = cv2.imread("sar_1.jpg", cv2.IMREAD_GRAYSCALE)

In [ ]:
# 1. Добавление шума
# Гауссовский шум
mean = 0
stddev = 100
noise_gauss = np.zeros(img_gray.shape, np.uint8)
cv2.randn(noise_gauss, mean, stddev)
image_noise_gauss = cv2.add(img_gray, noise_gauss)

# Постоянный шум
constant_value = 30
image_noise_const = np.clip(img_gray.astype(np.int16) + constant_value, 0, 255).astype(np.uint8)

plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.title("Оригинал")
plt.imshow(img_gray, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Гауссовский шум")
plt.imshow(image_noise_gauss, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Постоянный шум")
plt.imshow(image_noise_const, cmap="gray")
plt.axis("off")

plt.suptitle("2.0. Добавление шума (Гауссовский и Постоянный)", fontsize=14)
plt.tight_layout()
plt.show()

![Изображение](Figure_1.png)

In [ ]:
from skimage.metrics import mean_squared_error

# 2. Протестировать фильтры с различными параметрами
# Гауссовский шум
mean = 0
var = 100
sigma = var ** 0.5
gauss = np.random.normal(mean, sigma, img_gray.shape).astype(np.float32)
noisy = np.clip(img_gray.astype(np.float32) + gauss, 0, 255).astype(np.uint8)

results = [("Оригинал", img_gray),
           ("Зашумленное", noisy)]

# Медианный фильтр
for ksize in [3, 5, 7]:
    filtered = cv2.medianBlur(noisy, ksize)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results.append((f"Медианный {ksize}x{ksize}\nMSE={mse_val:.1f} SSIM={ssim_val:.3f}", filtered))

# Гауссов фильтр
for sigma in [0.5, 1.5, 3]:
    filtered = cv2.GaussianBlur(noisy, (5, 5), sigma)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results.append((f"Гауссов σ={sigma}\nMSE={mse_val:.1f} SSIM={ssim_val:.3f}", filtered))

# Билатериальный фильтр
params_bilateral = [(5, 50, 50), (9, 75, 75), (15, 150, 150)]
for (d, sC, sS) in params_bilateral:
    filtered = cv2.bilateralFilter(noisy, d, sC, sS)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results.append((f"Билатеральный d={d}\nMSE={mse_val:.1f} SSIM={ssim_val:.3f}", filtered))

# Нелокальные средние
params_nlm = [(5, 3, 15), (10, 7, 21), (20, 10, 31)]
for (h, t, w) in params_nlm:
    filtered = cv2.fastNlMeansDenoising(noisy, None, h, t, w)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results.append((f"Нелокальные h={h}\nMSE={mse_val:.1f} SSIM={ssim_val:.3f}", filtered))

plt.figure(figsize=(18, 10))
for i, (name, image) in enumerate(results, start=1):
    plt.subplot(4, 4, i)
    plt.imshow(image, cmap="gray")
    plt.title(name, fontsize=9)
    plt.axis("off")

plt.suptitle("2. Протестировать фильтры с различными параметрами (MSE и SSIM)", fontsize=16)
plt.tight_layout()
plt.show()

![Изображение](Figure_2.png)

In [ ]:
# 3. Выяснить, какой фильтр показал лучший результат фильтрации шума.
results_metrics = []

# Медианный фильтр
for ksize in [3, 5, 7]:
    filtered = cv2.medianBlur(noisy, ksize)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results_metrics.append(("Медианный", f"{ksize}x{ksize}", mse_val, ssim_val))

# Гауссов фильтр
for sigma in [0.5, 1.5, 3]:
    filtered = cv2.GaussianBlur(noisy, (5, 5), sigma)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results_metrics.append(("Гауссов", f"σ={sigma}", mse_val, ssim_val))

# Билатериальный фильтр
for (d, sC, sS) in [(5, 50, 50), (9, 75, 75), (15, 150, 150)]:
    filtered = cv2.bilateralFilter(noisy, d, sC, sS)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results_metrics.append(("Билатеральный", f"d={d}", mse_val, ssim_val))

# Нелокальные средние
for (h, t, w) in [(5, 3, 15), (10, 7, 21), (20, 10, 31)]:
    filtered = cv2.fastNlMeansDenoising(noisy, None, h, t, w)
    mse_val = mean_squared_error(img_gray, filtered)
    ssim_val = ssim(img_gray, filtered)
    results_metrics.append(("Нелокальные средние", f"h={h}", mse_val, ssim_val))

print(f"{'Фильтр':<20}{'Параметр':<12}{'MSE':<15}{'SSIM'}")
print("-" * 55)
for ftype, param, mse_val, ssim_val in results_metrics:
    print(f"{ftype:<20}{param:<12}{mse_val:<15.4f}{ssim_val:.4f}")

best_ssim = max(results_metrics, key=lambda x: x[3])
best_mse = min(results_metrics, key=lambda x: x[2])

print("\n Лучший по SSIM:")
print(f"{best_ssim[0]} фильтр ({best_ssim[1]}) → MSE={best_ssim[2]:.4f}, SSIM={best_ssim[3]:.4f}")

print("\n Лучший по MSE:")
print(f"{best_mse[0]} фильтр ({best_mse[1]}) → MSE={best_mse[2]:.4f}, SSIM={best_mse[3]:.4f}")

Фильтр              Параметр    MSE            SSIM
-------------------------------------------------------
Медианный           3x3         116.2438       0.7727
Медианный           5x5         203.0900       0.6159
Медианный           7x7         265.8181       0.5116
Гауссов             σ=0.5       55.9592        0.8808
Гауссов             σ=1.5       156.2086       0.7026
Гауссов             σ=3         193.0673       0.6320
Билатеральный       d=5         97.8091        0.7876
Билатеральный       d=9         204.6411       0.5852
Билатеральный       d=15        360.7810       0.3743
Нелокальные средние h=5         95.4685        0.8263
Нелокальные средние h=10        68.0588        0.8386
Нелокальные средние h=20        242.9282       0.5064

Лучший по SSIM:
Гауссов фильтр (σ=0.5) → MSE=55.9592, SSIM=0.8808

Лучший по MSE:
Гауссов фильтр (σ=0.5) → MSE=55.9592, SSIM=0.8808